#### This Version Creates the Graph in Neo4J then runs the Queries.

In [2]:
from neo4j import GraphDatabase
import os
import pandas as pd
import shutil
import subprocess

This moves the nodes and relationships files from data/current_working_graph into the DBMS import folder.

At the moment this uses local paths, but the rest should be automated.

In [4]:
# Define the source directory
source_dir = os.path.abspath('../../data/graph_1_9/train')

# Define the destination directory
# This should be the import directory of your graph database
destination_dir = '/home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-f6eddb81-f545-4b54-a638-c4e3be56c49c/import'

# Specify the filenames to move
filenames = ['nodes.csv', 'relationships.csv']

for filename in filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    
    # Move each file
    shutil.copy(source_path, destination_path)

Then run the admin import command.

In [8]:
# Define the working directory
working_dir = '/home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-f6eddb81-f545-4b54-a638-c4e3be56c49c'

# Construct the command
command = [
    './bin/neo4j-admin', 'database', 'import', 'full',
    '--nodes=import/nodes.csv',
    '--relationships=import/relationships.csv', 'neo4j'
]

#command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv neo4j'

# Execute the command

result = subprocess.run(command, cwd=working_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
b"Neo4j version: 5.12.0\nImporting the contents of these files into /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-f6eddb81-f545-4b54-a638-c4e3be56c49c/data/databases/neo4j:\nNodes:\n  /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-f6eddb81-f545-4b54-a638-c4e3be56c49c/import/nodes.csv\n\nRelationships:\n  /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-f6eddb81-f545-4b54-a638-c4e3be56c49c/import/relationships.csv\n\n\nAvailable resources:\n  Total machine memory: 15.47GiB\n  Free machine memory: 219.3MiB\n  Max heap memory : 910.5MiB\n  Max worker threads: 8\n  Configured max memory: 111.3MiB\n  High parallel IO: true\n\nCypher type normalization is enabled (disable with --normalize-types=false):\n  Property type of 'WaveNumber' normalized from 'float' --> 'double' in /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-f6eddb81-f545-4b54-a638-c4e3be56c49c/import/nodes.csv\n  Property typ

Run and connect to the Neo4j Database

In [12]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
#password = os.environ['NEO4J_Password']           # Neo4J password
password = '26622002'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [13]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['Normal', 'Hyperglycemia', 'Hypoglycemia'],
      {
        SAMPLE: {
          orientation: 'UNDIRECTED',
          properties: 'DIST'
        },
        GRID: {
          orientation: 'UNDIRECTED',
          properties: 'DIST'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: A graph with name 'myGraph' already exists.}

Define the graph algorithms.

In [14]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [15]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [16]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [17]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [18]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [19]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [20]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [14]:
def run_node2vec_algorithm(tx):
    query = """
    CALL gds.node2vec.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

In [21]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        { relationshipWeightProperty: 'DIST',
         randomSeed:1234,
         embeddingDimension: 128
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["embedding"]) for record in results]

Execute the algorithms and store the results in a Dataframe.

In [22]:
# Use a session to execute the queries and retrieve the results
with driver.session() as session:
    pagerank_results = session.execute_read(run_pagerank_centrality)
    degree_results = session.execute_read(run_degree_centrality)
    eigenvector_results = session.execute_read(run_eigenvector_centrality)
    articlerank_results = session.execute_read(run_articlerank_centrality)
    label_propagation_results = session.execute_read(run_label_propagation_algorithm)
    leiden_results = session.execute_read(run_leiden_algorithm)
    louvain_results = session.execute_read(run_louvain_algorithm)

In [23]:
pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'id', 'PageRank'])
degree_df = pd.DataFrame(degree_results, columns=['name', 'id', 'DegreeCentrality'])
eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'id', 'EigenvectorCentrality'])
articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'id', 'ArticleRank'])
label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'id', 'LabelPropagation'])
leiden_df = pd.DataFrame(leiden_results, columns=['name', 'id', 'Leiden'])
louvain_df = pd.DataFrame(louvain_results, columns=['name', 'id', 'Louvain'])

In [ ]:
pagerank_df = pagerank_df.groupby("name").mean().reset_index()

In [65]:
degree_df = degree_df.groupby("name").mean().reset_index()
eigenvector_df = eigenvector_df.groupby("name").mean().reset_index()
articlerank_df = articlerank_df.groupby("name").mean().reset_index()
eigenvector_df = eigenvector_df.groupby("name").mean().reset_index()
leiden_df = leiden_df.groupby("name").first().reset_index()
louvain_df = louvain_df.groupby("name").first().reset_index()

In [24]:
merged_df = pagerank_df
for df in [degree_df, eigenvector_df, articlerank_df, leiden_df, louvain_df]:
    merged_df = pd.merge(merged_df, df, on=['id', 'name'], how='left')

In [25]:
df = merged_df.rename(columns={'name' : 'SpecID'})
df

,SpecID,id,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain
0,210407-2-08,5289652,1.487887,543.682362,0.008894,0.298572,46,31753
1,210304-2-20,2554528,1.466770,299.735350,0.008258,0.236928,8,23569
2,210304-2-16,2543947,1.462001,268.882009,0.008094,0.228633,0,23569
3,210304-2-16,2543999,1.460277,268.982704,0.008088,0.228650,0,23569
4,210407-2-08,5289244,1.447841,500.355360,0.008408,0.288114,46,31594
...,...,...,...,...,...,...,...,...
41539,201210-1-06,18353,0.216328,23.347484,0.000296,0.156496,45,16100
41540,201210-1-09,23620,0.214901,18.060851,0.000213,0.155111,19,31642
41541,210114-2-34,484813,0.214737,32.948812,0.000389,0.158864,45,1277
41542,210414-3-34,5607088,0.213060,19.766993,0.000296,0.155656,46,31594


In [26]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SpecID', 'id', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SpecID,id,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-1-00,613,0.139784,0.051968,0.070142,0.299591,-0.063975,0.263664,0.067435,0.234340,...,-0.194144,0.095186,-0.006200,0.081355,0.119563,-0.133779,-0.141744,-0.068619,0.204436,0.199861
1,201210-1-00,727,0.274053,0.007118,0.351328,-0.206510,0.197247,0.119952,-0.155315,-0.020098,...,0.235635,0.172471,0.026762,0.010667,-0.068112,0.205550,-0.056949,-0.238111,-0.132707,-0.209549
2,201210-1-00,1026,-0.119437,-0.051044,0.005381,-0.362583,0.106320,0.384810,-0.085303,-0.314473,...,-0.220398,0.135333,0.373200,0.179441,-0.214181,0.138212,-0.067846,-0.057725,-0.000133,-0.233276
3,201210-1-00,1165,-0.012199,0.080668,-0.102105,-0.158632,-0.313209,0.225440,-0.410707,-0.008687,...,-0.167724,0.059866,0.108602,0.170323,0.108949,0.063636,0.076978,0.057380,-0.028786,0.054376
4,201210-1-00,1964,-0.065723,-0.013451,-0.227100,-0.036800,-0.050909,0.044256,0.224032,0.036915,...,0.067838,0.331496,-0.178084,-0.222299,0.048845,0.020869,0.070090,0.300467,0.240191,-0.129091


In [27]:
fastRP_df

,SpecID,id,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-1-00,613,0.139784,0.051968,0.070142,0.299591,-0.063975,0.263664,0.067435,0.234340,...,-0.194144,0.095186,-0.006200,0.081355,0.119563,-0.133779,-0.141744,-0.068619,0.204436,0.199861
1,201210-1-00,727,0.274053,0.007118,0.351328,-0.206510,0.197247,0.119952,-0.155315,-0.020098,...,0.235635,0.172471,0.026762,0.010667,-0.068112,0.205550,-0.056949,-0.238111,-0.132707,-0.209549
2,201210-1-00,1026,-0.119437,-0.051044,0.005381,-0.362583,0.106320,0.384810,-0.085303,-0.314473,...,-0.220398,0.135333,0.373200,0.179441,-0.214181,0.138212,-0.067846,-0.057725,-0.000133,-0.233276
3,201210-1-00,1165,-0.012199,0.080668,-0.102105,-0.158632,-0.313209,0.225440,-0.410707,-0.008687,...,-0.167724,0.059866,0.108602,0.170323,0.108949,0.063636,0.076978,0.057380,-0.028786,0.054376
4,201210-1-00,1964,-0.065723,-0.013451,-0.227100,-0.036800,-0.050909,0.044256,0.224032,0.036915,...,0.067838,0.331496,-0.178084,-0.222299,0.048845,0.020869,0.070090,0.300467,0.240191,-0.129091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41539,210526-3-49,8008132,-0.052659,0.133898,0.010714,-0.235280,0.104910,0.196440,-0.026976,0.000628,...,0.234500,-0.161010,-0.060033,0.081107,-0.096745,-0.059125,0.125223,-0.130566,0.149208,0.097632
41540,210526-3-49,8008234,0.081748,0.164308,-0.032551,0.045489,0.235424,0.215085,0.105911,-0.396822,...,0.203814,-0.189690,-0.214241,-0.002678,0.230862,-0.042159,0.088526,0.037827,0.059302,0.015662
41541,210526-3-49,8008504,-0.072395,0.474383,-0.151729,0.142953,0.039449,0.050774,0.132637,-0.178333,...,-0.174071,-0.042864,0.087330,0.190069,0.199276,-0.104089,-0.072591,-0.135590,0.118115,0.069720
41542,210526-3-49,8008936,-0.076270,0.072457,-0.158693,-0.233478,0.014417,-0.208415,0.109368,-0.010691,...,-0.393790,0.130154,-0.232905,0.342496,-0.112222,-0.037431,0.342837,0.210332,-0.531102,0.219559


Delete the projection

In [87]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)

In [88]:
close_driver()

In [58]:
status_df

,SpecID,WaveNumber:float,Absorbance:float,Status:LABEL,Widths:float,Prominences:float,GridSlot,Node_ID:ID
0,201210-1-00,618.90662,1842.5710,Normal,57.732793,181.9741,"(12, 1)",613
1,201210-1-00,696.81091,1851.9185,Normal,33.525178,193.5749,"(13, 1)",727
2,201210-1-00,901.13898,1746.4041,Normal,107.305604,157.9830,"(18, 1)",1026
3,201210-1-00,996.12756,1702.7238,Normal,26.861586,115.9938,"(19, 1)",1165
4,201210-1-00,1542.14120,1475.0056,Normal,15.749470,87.6968,"(30, 1)",1964
...,...,...,...,...,...,...,...,...
51922,210526-3-49,450.79727,1482.5781,Hyperglycemia,11.954149,98.2019,"(9, 1)",8008132
51923,210526-3-49,520.50116,1482.3167,Hyperglycemia,13.875621,86.4581,"(10, 1)",8008234
51924,210526-3-49,705.01141,1507.8912,Hyperglycemia,54.718502,125.7380,"(14, 1)",8008504
51925,210526-3-49,1000.22780,1505.2144,Hyperglycemia,18.241825,110.7786,"(20, 1)",8008936


In [30]:
status_df = pd.read_csv('../../data/graph_1_9/train/nodes.csv')

In [31]:
fastRP_df['id'] = fastRP_df['id'].astype(int)

In [32]:
df['id'] = df['id'].astype(int)

In [33]:
graph_df = pd.merge(df, status_df, left_on="id", right_on="Node_ID:ID", how="inner")
graph_df.drop(columns=["Node_ID:ID"], inplace=True)

In [34]:
fast_graph_df = pd.merge(fastRP_df, status_df, left_on="id", right_on="Node_ID:ID", how="inner")
fast_graph_df.drop(columns=["Node_ID:ID"], inplace=True)

In [35]:
graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)
graph_df.drop(columns=['SpecID_y','Absorbance:float', 'Widths:float', 'Prominences:float', 'GridSlot'], inplace=True)
graph_df.rename(columns={'SpecID_x':'SpecID'}, inplace=True)

In [36]:
fast_graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [37]:
fast_graph_df.drop(columns=['SpecID_y', 'WaveNumber:float', 'Absorbance:float', 'Widths:float', 'Prominences:float', 'GridSlot'], inplace=True)
fast_graph_df.rename(columns={'SpecID_x':'SpecID'}, inplace=True)

In [38]:
graph_df

,SpecID,id,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain,WaveNumber:float,Status
0,210407-2-08,5289652,1.487887,543.682362,0.008894,0.298572,46,31753,1024.82910,Normal
1,210304-2-20,2554528,1.466770,299.735350,0.008258,0.236928,8,23569,1028.92940,Hypoglycemia
2,210304-2-16,2543947,1.462001,268.882009,0.008094,0.228633,0,23569,1000.91110,Hypoglycemia
3,210304-2-16,2543999,1.460277,268.982704,0.008088,0.228650,0,23569,1036.44640,Hypoglycemia
4,210407-2-08,5289244,1.447841,500.355360,0.008408,0.288114,46,31594,746.01367,Normal
...,...,...,...,...,...,...,...,...,...,...
41539,201210-1-06,18353,0.216328,23.347484,0.000296,0.156496,45,16100,1937.81320,Normal
41540,201210-1-09,23620,0.214901,18.060851,0.000213,0.155111,19,31642,1935.76310,Normal
41541,210114-2-34,484813,0.214737,32.948812,0.000389,0.158864,45,1277,1982.23230,Normal
41542,210414-3-34,5607088,0.213060,19.766993,0.000296,0.155656,46,31594,1869.47610,Normal


In [39]:
# Define a function to calculate the bin for a given wavenumber with a specified bin size
def calculate_bin_interval(wavenumber, bin_size):
    bin_start = int((wavenumber - 200) / bin_size) * bin_size + 200
    bin_end = bin_start + bin_size
    return f"{bin_start}-{bin_end}"

# Set the bin size
bin_size = 25

# Add a "Bin" column to the DataFrame
graph_df['Bin'] = graph_df['WaveNumber:float'].apply(lambda x: calculate_bin_interval(x, bin_size))

In [40]:
# Pivot table with 'Absorbance', 'PeakWidths', and 'PeakProminences' as values
peak_bins = graph_df.pivot_table(index='SpecID', columns='Bin', values=['PageRank', 'DegreeCentrality', 'EigenvectorCentrality', 'ArticleRank', 'Leiden', 'Louvain'], aggfunc='mean')
peak_bins.columns = [f"{col[0]}_{col[1]}" for col in peak_bins.columns]  # Combine column names
peak_bins.reset_index(inplace=True)

# Merge with 'Status' information
statuses = graph_df[['SpecID', 'Status']].drop_duplicates()
peak_bins = pd.merge(peak_bins, statuses, on='SpecID')

# Set 'SpecID' as the index
peak_bins.set_index('SpecID', inplace=True)

# Fill NaN values with False
peak_bins.fillna(False, inplace=True)

In [41]:
graph_bins = peak_bins.reset_index()

In [42]:
peak_bins

,ArticleRank_1000-1025,ArticleRank_1025-1050,ArticleRank_1050-1075,ArticleRank_1075-1100,ArticleRank_1100-1125,ArticleRank_1125-1150,ArticleRank_1150-1175,ArticleRank_1175-1200,ArticleRank_1200-1225,ArticleRank_1225-1250,...,PageRank_775-800,PageRank_800-825,PageRank_825-850,PageRank_850-875,PageRank_875-900,PageRank_900-925,PageRank_925-950,PageRank_950-975,PageRank_975-1000,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,0.658227,False,False,0.914623,Normal
201210-1-01,False,False,False,False,False,0.283161,False,False,False,0.301772,...,False,False,False,False,0.838032,False,False,False,0.829868,Normal
201210-1-02,0.343787,False,False,False,False,0.272106,False,False,False,False,...,False,False,False,False,False,0.647596,False,False,False,Normal
201210-1-03,False,False,0.230439,False,False,False,0.213949,False,False,0.298788,...,False,False,False,False,False,0.722791,False,False,0.807854,Normal
201210-1-04,0.337577,False,False,False,False,False,False,0.262892,False,False,...,False,False,False,False,False,False,False,False,False,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210526-3-45,0.355752,False,False,False,0.29417,False,False,False,False,False,...,False,False,False,1.068007,False,False,False,False,False,Hyperglycemia
210526-3-46,0.362974,False,False,False,False,0.297343,False,False,False,False,...,False,False,False,1.093228,False,False,False,False,False,Hyperglycemia
210526-3-47,0.367732,False,False,False,0.300111,False,False,False,False,False,...,False,False,False,False,False,0.722534,False,False,False,Hyperglycemia


In [59]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Splitting the dataframe into features (X) and target variable (y)
X = peak_bins.drop(['Status'], axis=1)
y = peak_bins['Status']

# Creating the classifiers
rf = RandomForestClassifier(random_state=1234)
et = ExtraTreesClassifier(random_state=1234)

# Combining the classifiers into a list
classifiers = [rf, et]

# Performing 10-fold cross-validation for each classifier
for clf in classifiers:
    
    # Using StratifiedKFold for classification tasks
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
    
    # Getting cross-validation scores
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Displaying the results
    print(f'{clf.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

RandomForestClassifier Cross-Validation Accuracy: 0.7406 +/- 0.0352
ExtraTreesClassifier Cross-Validation Accuracy: 0.7499 +/- 0.0267


In [60]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
graph_rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
graph_rf_model.fit(X_train, y_train)

# Extra Trees model
graph_et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
graph_et_model.fit(X_train, y_train)

# Predictions
rf_predictions = graph_rf_model.predict(X_test)
et_predictions = graph_et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

In [45]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [46]:
def get_feature_importances(model, X):

    # Get feature importances
    feature_importances = model.feature_importances_

    # Creating a DataFrame to display feature importances
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

    # Sorting the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Show the top 10 most important features
    top_10_features = feature_importance_df.head(10)

    return top_10_features

In [47]:
calculate_metrics(y_test, rf_predictions)

Overall Accuracy: 0.710691823899371


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.78      0.61      0.68       152
 Hypoglycemia       0.66      0.78      0.71       165
       Normal       0.73      0.74      0.73       160

     accuracy                           0.71       477
    macro avg       0.72      0.71      0.71       477
 weighted avg       0.72      0.71      0.71       477


Confusion Matrix:
[[ 93  38  21]
 [ 14 128  23]
 [ 13  29 118]]


In [48]:
calculate_metrics(y_test, et_predictions)

Overall Accuracy: 0.7232704402515723


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.79      0.63      0.70       152
 Hypoglycemia       0.67      0.82      0.74       165
       Normal       0.73      0.71      0.72       160

     accuracy                           0.72       477
    macro avg       0.73      0.72      0.72       477
 weighted avg       0.73      0.72      0.72       477


Confusion Matrix:
[[ 96  33  23]
 [ 11 136  18]
 [ 14  33 113]]


In [61]:
get_feature_importances(graph_rf_model, X)

,Feature,Importance
72,DegreeCentrality_1000-1025,0.010568
0,ArticleRank_1000-1025,0.010037
402,PageRank_250-275,0.009209
42,ArticleRank_250-275,0.009035
114,DegreeCentrality_250-275,0.009007
186,EigenvectorCentrality_250-275,0.008902
144,EigenvectorCentrality_1000-1025,0.008265
330,Louvain_250-275,0.008262
57,ArticleRank_625-650,0.008161
153,EigenvectorCentrality_1225-1250,0.007628


In [62]:
get_feature_importances(graph_et_model, X)

,Feature,Importance
72,DegreeCentrality_1000-1025,0.007336
0,ArticleRank_1000-1025,0.007066
402,PageRank_250-275,0.006929
124,DegreeCentrality_500-525,0.006928
42,ArticleRank_250-275,0.006751
330,Louvain_250-275,0.006363
67,ArticleRank_875-900,0.006345
216,Leiden_1000-1025,0.006344
153,EigenvectorCentrality_1225-1250,0.006169
258,Leiden_250-275,0.006080


## FastRp

In [51]:
fast_graph_df.drop(columns=['id'], inplace=True)
fast_graph_df = fast_graph_df.groupby(['SpecID', 'Status']).max().reset_index()

In [64]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = fast_graph_df.drop(columns=['SpecID', 'Status'])

# Target variable
y = fast_graph_df['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest model
fastrp_rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
fastrp_rf_model.fit(X_train, y_train)

# Extra Trees model
fastrp_et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
fastrp_et_model.fit(X_train, y_train)

# Predictions
rf_predictions = fastrp_rf_model.predict(X_test)
et_predictions = fastrp_et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.6876310272536688
Extra Trees Accuracy: 0.7169811320754716


Fast rp mixed with traditional graph metrics

In [53]:
joined_graph = pd.merge(graph_bins, fast_graph_df, on=['SpecID', 'Status'], how='inner')

In [54]:
joined_graph

,SpecID,ArticleRank_1000-1025,ArticleRank_1025-1050,ArticleRank_1050-1075,ArticleRank_1075-1100,ArticleRank_1100-1125,ArticleRank_1125-1150,ArticleRank_1150-1175,ArticleRank_1175-1200,ArticleRank_1200-1225,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-1-00,False,False,False,False,False,False,False,False,False,...,0.235635,0.331496,0.373200,0.179441,0.119563,0.205550,0.076978,0.300467,0.240191,0.199861
1,201210-1-01,False,False,False,False,False,0.283161,False,False,False,...,0.264341,0.123945,0.190787,0.337781,0.183891,0.200432,0.179221,0.361687,0.279632,0.048672
2,201210-1-02,0.343787,False,False,False,False,0.272106,False,False,False,...,0.218275,0.103757,0.180941,0.357337,0.306508,0.041576,0.132432,0.342729,0.233369,0.158262
3,201210-1-03,False,False,0.230439,False,False,False,0.213949,False,False,...,0.022077,0.406188,0.201945,0.366354,0.253276,0.192682,0.149321,0.229277,0.267769,0.425681
4,201210-1-04,0.337577,False,False,False,False,False,False,0.262892,False,...,0.133203,0.119515,0.187896,0.352588,0.230987,0.033428,0.149142,0.361101,0.244995,0.032651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,210526-3-45,0.355752,False,False,False,0.29417,False,False,False,False,...,0.277104,0.128988,0.273230,0.339282,0.233841,0.295253,0.356312,0.201762,0.182468,0.211821
2379,210526-3-46,0.362974,False,False,False,False,0.297343,False,False,False,...,0.209568,0.129594,0.088118,0.351371,0.233342,0.313950,0.352198,0.204217,0.180794,0.213872
2380,210526-3-47,0.367732,False,False,False,0.300111,False,False,False,False,...,0.236841,0.175049,0.348880,0.353697,0.227350,0.316925,0.349833,0.204928,0.178302,0.215728
2381,210526-3-48,0.370998,False,False,False,False,0.300504,False,False,False,...,0.236968,0.177182,0.047626,0.347244,0.231318,0.318081,0.344218,0.208359,0.176914,0.217601


In [55]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = joined_graph.drop(columns=['SpecID', 'Status'])

# Target variable
y = joined_graph['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.710691823899371
Extra Trees Accuracy: 0.7358490566037735


## Full Graph

In [57]:
# Define the source directory
source_dir = os.path.abspath('../../data/graph_1_9/full')

# Define the destination directory
# This should be the import directory of your graph database
destination_dir = '/home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-39a2207a-43dc-48d1-a88e-c8937fb7935a/import'

# Specify the filenames to move
filenames = ['nodes.csv', 'relationships.csv']

for filename in filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    
    # Move each file
    shutil.copy(source_path, destination_path)

Then run the admin import command.

In [58]:
# Define the working directory
working_dir = '/home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-39a2207a-43dc-48d1-a88e-c8937fb7935a'

# Construct the command
command = [
    './bin/neo4j-admin', 'database', 'import', 'full',
    '--nodes=import/nodes.csv',
    '--relationships=import/relationships.csv', 'neo4j'
]

#command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv neo4j'

# Execute the command


result = subprocess.run(command, cwd=working_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
b"Neo4j version: 5.12.0\nImporting the contents of these files into /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-39a2207a-43dc-48d1-a88e-c8937fb7935a/data/databases/neo4j:\nNodes:\n  /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-39a2207a-43dc-48d1-a88e-c8937fb7935a/import/nodes.csv\n\nRelationships:\n  /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-39a2207a-43dc-48d1-a88e-c8937fb7935a/import/relationships.csv\n\n\nAvailable resources:\n  Total machine memory: 15.47GiB\n  Free machine memory: 197.3MiB\n  Max heap memory : 910.5MiB\n  Max worker threads: 8\n  Configured max memory: 101.5MiB\n  High parallel IO: true\n\nCypher type normalization is enabled (disable with --normalize-types=false):\n  Property type of 'WaveNumber' normalized from 'float' --> 'double' in /home/stang/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-39a2207a-43dc-48d1-a88e-c8937fb7935a/import/nodes.csv\n  Property typ

Run and connect to the Neo4j Database

In [65]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
#password = os.environ['NEO4J_Password']           # Neo4J password
password = '26622002'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [66]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['Normal', 'Hyperglycemia', 'Hypoglycemia'],
      {
        SAMPLE: {
          orientation: 'UNDIRECTED',
          properties: 'DIST'
        },
        GRID: {
          orientation: 'UNDIRECTED',
          properties: 'DIST'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

Define the graph algorithms.

In [67]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [68]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [71]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [72]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [73]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [74]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [75]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [ ]:
def run_node2vec_algorithm(tx):
    query = """
    CALL gds.node2vec.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

In [76]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        { relationshipWeightProperty: 'DIST',
         randomSeed:1234,
         embeddingDimension: 128
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["embedding"]) for record in results]

Execute the algorithms and store the results in a Dataframe.

In [77]:
# Use a session to execute the queries and retrieve the results
with driver.session() as session:
    pagerank_results = session.execute_read(run_pagerank_centrality)
    degree_results = session.execute_read(run_degree_centrality)
    eigenvector_results = session.execute_read(run_eigenvector_centrality)
    articlerank_results = session.execute_read(run_articlerank_centrality)
    label_propagation_results = session.execute_read(run_label_propagation_algorithm)
    leiden_results = session.execute_read(run_leiden_algorithm)
    louvain_results = session.execute_read(run_louvain_algorithm)

In [78]:
pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'id', 'PageRank'])
degree_df = pd.DataFrame(degree_results, columns=['name', 'id', 'DegreeCentrality'])
eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'id', 'EigenvectorCentrality'])
articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'id', 'ArticleRank'])
label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'id', 'LabelPropagation'])
leiden_df = pd.DataFrame(leiden_results, columns=['name', 'id', 'Leiden'])
louvain_df = pd.DataFrame(louvain_results, columns=['name', 'id', 'Louvain'])

In [79]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SpecID', 'id', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SpecID,id,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,210504-3-30,6558989,-0.033804,0.095718,-0.005400,0.065173,-0.014176,0.181891,-0.070871,0.095966,...,-0.080663,0.032825,0.227665,0.173248,0.088141,-0.032875,-0.311489,-0.140437,-0.127138,0.278497
1,210504-3-30,6559028,0.222140,-0.060719,-0.178434,0.148498,-0.006308,0.123289,-0.181790,-0.190075,...,0.044288,-0.069051,0.082455,0.195629,-0.069366,0.006223,-0.288481,-0.194194,-0.061998,0.246965
2,210504-3-30,6559055,0.073465,0.104231,-0.055980,0.072610,0.048931,0.170230,-0.009979,-0.120915,...,0.065912,-0.338712,0.220771,0.466374,-0.043528,0.009926,-0.238907,-0.042726,-0.008282,0.149076
3,210504-3-30,6559126,-0.005119,0.062323,-0.096963,0.050637,-0.002927,0.143369,-0.140663,-0.143405,...,0.084532,0.012804,0.134871,0.110892,0.090382,0.122019,-0.325879,-0.140899,-0.077806,0.245083
4,210504-3-30,6559146,-0.003811,0.058733,-0.095958,0.051626,0.001289,0.140344,-0.135946,-0.145829,...,0.071677,0.014869,0.126632,0.101104,0.094937,0.126524,-0.320751,-0.135510,-0.073486,0.252146


In [81]:
merged_df = pagerank_df
for df in [degree_df, eigenvector_df, articlerank_df, leiden_df, louvain_df]:
    merged_df = pd.merge(merged_df, df, on=['id', 'name'], how='left')
df = merged_df.rename(columns={'name' : 'SpecID'})
df

,SpecID,id,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain
0,210304-2-20,2554528,1.472942,346.263893,0.007547,0.232032,2,479
1,210407-2-08,5289652,1.458982,684.913033,0.007900,0.301263,1,17979
2,210304-2-42,2612504,1.434791,341.089958,0.007298,0.230766,2,479
3,210304-2-16,2543947,1.429760,290.571562,0.007041,0.219674,2,10190
4,210304-2-16,2543999,1.427356,290.504637,0.007031,0.219649,2,10190
...,...,...,...,...,...,...,...,...
51922,201210-1-22,57890,0.214485,25.412934,0.000221,0.155686,26,16940
51923,210114-1-38,361035,0.211477,9.794399,0.000170,0.152430,7,13914
51924,210114-2-34,484813,0.202604,32.973024,0.000282,0.157219,6,39991
51925,210414-3-34,5607088,0.202541,19.781042,0.000215,0.154603,1,31970


In [82]:
status_df = pd.read_csv('../../data/graph_1_9/test/nodes.csv')
fastRP_df['id'] = fastRP_df['id'].astype(int)
df['id'] = df['id'].astype(int)
graph_df = pd.merge(df, status_df, left_on="id", right_on="Node_ID:ID", how="inner")
graph_df.drop(columns=["Node_ID:ID"], inplace=True)
fast_graph_df = pd.merge(fastRP_df, status_df, left_on="id", right_on="Node_ID:ID", how="inner")
fast_graph_df.drop(columns=["Node_ID:ID"], inplace=True)
graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)
graph_df.drop(columns=['SpecID_y','Absorbance:float', 'Widths:float', 'Prominences:float', 'GridSlot'], inplace=True)
graph_df.rename(columns={'SpecID_x':'SpecID'}, inplace=True)
fast_graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)
fast_graph_df.drop(columns=['SpecID_y', 'WaveNumber:float', 'Absorbance:float', 'Widths:float', 'Prominences:float', 'GridSlot'], inplace=True)
fast_graph_df.rename(columns={'SpecID_x':'SpecID'}, inplace=True)
graph_df

,SpecID,id,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain,WaveNumber:float,Status
0,210211-1-23,1113212,1.423764,449.219480,0.007742,0.253500,9,479,1048.7472,Hyperglycemia
1,210304-2-38,2601917,1.394532,336.133549,0.007116,0.229523,2,479,1000.9111,Hypoglycemia
2,210121-1-44,905015,1.393451,333.127617,0.007152,0.228833,2,479,1026.8793,Hyperglycemia
3,210304-2-45,2620362,1.383726,335.208963,0.007055,0.229295,2,479,1000.9111,Hypoglycemia
4,210310-1-17,2678378,1.383685,332.236744,0.007097,0.228594,2,479,1032.3462,Normal
...,...,...,...,...,...,...,...,...,...,...
10378,210428-2-03,5952427,0.268211,31.918515,0.000480,0.157528,1,17979,1974.7152,Normal
10379,210414-2-25,5715100,0.266134,39.363263,0.000552,0.159185,0,31759,1853.7585,Hypoglycemia
10380,210324-1-25,3754499,0.263439,53.491485,0.000446,0.161989,5,29296,1743.7357,Normal
10381,210415-1-41,4574164,0.248492,30.367754,0.000412,0.157114,0,31759,1866.7426,Hypoglycemia


In [86]:
# Define a function to calculate the bin for a given wavenumber with a specified bin size
def calculate_bin_interval(wavenumber, bin_size):
    bin_start = int((wavenumber - 200) / bin_size) * bin_size + 200
    bin_end = bin_start + bin_size
    return f"{bin_start}-{bin_end}"

# Set the bin size
bin_size = 25

# Add a "Bin" column to the DataFrame
graph_df['Bin'] = graph_df['WaveNumber:float'].apply(lambda x: calculate_bin_interval(x, bin_size))
# Pivot table with 'Absorbance', 'PeakWidths', and 'PeakProminences' as values
peak_bins = graph_df.pivot_table(index='SpecID', columns='Bin', values=['PageRank', 'DegreeCentrality', 'EigenvectorCentrality', 'ArticleRank', 'Leiden', 'Louvain'], aggfunc='mean')
peak_bins.columns = [f"{col[0]}_{col[1]}" for col in peak_bins.columns]  # Combine column names
peak_bins.reset_index(inplace=True)

# Merge with 'Status' information
statuses = graph_df[['SpecID', 'Status']].drop_duplicates()
peak_bins = pd.merge(peak_bins, statuses, on='SpecID')

# Set 'SpecID' as the index
peak_bins.set_index('SpecID', inplace=True)

# Fill NaN values with False
peak_bins.fillna(False, inplace=True)
graph_bins = peak_bins.reset_index()

In [87]:
X_test = peak_bins.drop(['Status'], axis=1)
y_test = peak_bins['Status']

# Predictions
rf_predictions = graph_rf_model.predict(X_test)
et_predictions = graph_et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

In [88]:
calculate_metrics(y_test, rf_predictions)

Overall Accuracy: 0.6879194630872483


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.72      0.63      0.67       195
 Hypoglycemia       0.65      0.74      0.69       196
       Normal       0.70      0.70      0.70       205

     accuracy                           0.69       596
    macro avg       0.69      0.69      0.69       596
 weighted avg       0.69      0.69      0.69       596


Confusion Matrix:
[[122  37  36]
 [ 26 145  25]
 [ 21  41 143]]


In [89]:
calculate_metrics(y_test, et_predictions)

Overall Accuracy: 0.6694630872483222


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.68      0.63      0.65       195
 Hypoglycemia       0.62      0.70      0.66       196
       Normal       0.72      0.68      0.70       205

     accuracy                           0.67       596
    macro avg       0.67      0.67      0.67       596
 weighted avg       0.67      0.67      0.67       596


Confusion Matrix:
[[122  44  29]
 [ 34 137  25]
 [ 24  41 140]]


In [90]:
fast_graph_df.drop(columns=['id'], inplace=True)
fast_graph_df = fast_graph_df.groupby(['SpecID', 'Status']).max().reset_index()

In [91]:
# Drop the SpecID column
X_test = fast_graph_df.drop(columns=['SpecID', 'Status'])

# Target variable
y_test = fast_graph_df['Status']

# Predictions
rf_predictions = fastrp_rf_model.predict(X_test)
et_predictions = fastrp_et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.3523489932885906
Extra Trees Accuracy: 0.3859060402684564


In [92]:
joined_graph = pd.merge(graph_bins, fast_graph_df, on=['SpecID', 'Status'], how='inner')

In [93]:
joined_graph

,SpecID,ArticleRank_1000-1025,ArticleRank_1025-1050,ArticleRank_1050-1075,ArticleRank_1075-1100,ArticleRank_1100-1125,ArticleRank_1125-1150,ArticleRank_1150-1175,ArticleRank_1175-1200,ArticleRank_1200-1225,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-1-10,0.327194,False,False,False,False,False,False,False,False,...,0.174145,0.306576,0.190821,0.293303,0.208554,0.228081,0.225902,0.219529,0.189622,0.382869
1,201210-1-12,0.319221,False,False,False,False,False,False,False,False,...,0.129365,0.096373,0.183873,0.171630,0.031642,0.288138,0.226378,0.217474,0.089820,0.176072
2,201210-1-14,0.310665,False,False,False,False,False,0.235115,False,False,...,0.168452,0.109232,0.415056,0.370557,0.238806,0.286417,0.322876,0.215117,0.257098,0.355038
3,201210-1-21,False,False,False,False,False,False,False,False,False,...,0.335456,0.152292,0.261854,0.099105,0.160350,0.286880,0.234223,0.339082,0.188879,0.155013
4,201210-1-22,0.355218,False,False,False,False,0.281222,False,False,False,...,0.334157,0.325833,0.061598,0.354299,0.253228,0.284353,0.239339,0.207170,0.249457,0.348569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,210526-3-05,0.343468,False,False,0.240963,False,0.246008,0.241891,False,0.284997,...,0.340696,0.236452,0.404684,0.335716,0.270184,0.279762,0.330756,0.199885,0.258199,0.351869
592,210526-3-06,0.349153,False,False,0.24265,False,0.264708,False,False,0.296188,...,0.341664,0.324421,0.227505,0.334419,0.267027,0.289566,0.332709,0.362922,0.236892,0.335285
593,210526-3-10,0.349359,False,False,False,False,False,False,0.268924,False,...,0.332579,0.135524,0.417246,0.360561,0.247998,0.284100,0.236567,0.292642,0.252750,0.414588
594,210526-3-16,False,False,False,False,False,False,0.269604,False,False,...,0.139024,0.115723,0.195422,0.201630,0.176378,0.275663,0.159476,0.283438,0.249698,0.044350


In [ ]:
# Drop the SpecID column
X_test = joined_graph.drop(columns=['SpecID', 'Status'])

# Target variable
y_test = joined_graph['Status']


# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)